# Project #15: Twitter hate speech detection 2

## 1. Introduction



The goal of our project is to find efficient methods for identifying hate speech on twitter. Our aim is to find a set of features that could be used to identify hate speech content.



For our analysis, we have gathered two data sets. The first data set was collected by searching for tweets containing specific hashtags (topics). The second data set was collected from active twitter users that frequently posted hate speech content. Both data sets were obtained using Twitter API and the search-tweets pytho library.

## Data sets

### Data-set 1: Hate speech hash tags
The first data set was collected by searching for tweets containing specific hashtags that were provided to us in the project assignment. The hash tags were: #terrorist, #radicalist, #islamophobia, #extremist, and #bombing. 
